# Digest Object Detection try
## Model preparation
### Variables

In [1]:
from pathlib import Path


# What model to download.
MODEL_NAME = "faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco_2018_01_28"
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = Path("__file__").parent.absolute().parent.joinpath("models").joinpath("research").joinpath(
    "object_detection").joinpath("data").joinpath("oid_bbox_trainable_label_map.pbtxt")

### Download Model

In [2]:
import urllib


opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

('faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco_2018_01_28.tar.gz',
 <http.client.HTTPMessage at 0x24345a7ca08>)

### Extract Files

In [3]:
import tarfile


tar_file = tarfile.open(MODEL_FILE)
tar_file.extractall()

### Load a (frozen) Tensorflow model into memory.

In [4]:
import tensorflow


detection_graph = tensorflow.Graph()
with detection_graph.as_default():
    od_graph_def = tensorflow.GraphDef()
    with tensorflow.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, "rb") as g_file:
        serialized_graph = g_file.read()
        od_graph_def.ParseFromString(serialized_graph)
        tensorflow.import_graph_def(od_graph_def, name="")

### Loading label map

In [5]:
from object_detection.utils import label_map_util



category_index = label_map_util.create_category_index_from_labelmap(str(PATH_TO_LABELS.absolute()), use_display_name=True)
category_index

{1: {'id': 1, 'name': 'Person'},
 2: {'id': 2, 'name': 'Clothing'},
 3: {'id': 3, 'name': 'Man'},
 4: {'id': 4, 'name': 'Face'},
 5: {'id': 5, 'name': 'Tree'},
 6: {'id': 6, 'name': 'Plant'},
 7: {'id': 7, 'name': 'Woman'},
 8: {'id': 8, 'name': 'Vehicle'},
 9: {'id': 9, 'name': 'Building'},
 10: {'id': 10, 'name': 'Land vehicle'},
 11: {'id': 11, 'name': 'Footwear'},
 12: {'id': 12, 'name': 'Girl'},
 13: {'id': 13, 'name': 'Animal'},
 14: {'id': 14, 'name': 'Car'},
 15: {'id': 15, 'name': 'Food'},
 16: {'id': 16, 'name': 'Wheel'},
 17: {'id': 17, 'name': 'Flower'},
 18: {'id': 18, 'name': 'Furniture'},
 19: {'id': 19, 'name': 'Window'},
 20: {'id': 20, 'name': 'House'},
 21: {'id': 21, 'name': 'Boy'},
 22: {'id': 22, 'name': 'Fashion accessory'},
 23: {'id': 23, 'name': 'Table'},
 24: {'id': 24, 'name': 'Glasses'},
 25: {'id': 25, 'name': 'Suit'},
 26: {'id': 26, 'name': 'Auto part'},
 27: {'id': 27, 'name': 'Bird'},
 28: {'id': 28, 'name': 'Sports equipment'},
 29: {'id': 29, 'name':

### Helper code

In [6]:
import numpy

from PIL import Image


def load_image_into_numpy_array(image: Image) -> numpy.ndarray:
    (im_width, im_height) = image.size
    array_image = numpy.array(image.getdata())
    if (len(array_image.shape) < 2) or (array_image.shape[1] > 3):
        image = image.convert("RGB")
        array_image = numpy.array(image.getdata())
    return array_image.reshape((im_height, im_width, 3)).astype(numpy.uint8)

### Classification code

In [7]:
import pandas



def label_image(path_to_image: Path) -> pandas.DataFrame:
    with tensorflow.Session(graph=detection_graph) as sess:
        ops = detection_graph.get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes']:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = detection_graph.get_tensor_by_name(tensor_name)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        image = Image.open(str(path_to_image))
        image_np = load_image_into_numpy_array(image)
        image_np_expanded = numpy.expand_dims(image_np, axis=0)
        output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image_np_expanded})
    output_frame = pandas.DataFrame()
    output_frame['detection_classes'] = output_dict['detection_classes'][0].astype(numpy.int64)
    output_frame['detection_scores'] = output_dict['detection_scores'][0]
    output_frame = output_frame[(output_frame["detection_scores"] > 0.65) & (output_frame['detection_classes'].isin([32, 203,
                284, 8, 14, 16, 26, 88, 87, 93, 102, 107, 214, 251, 436, 446, 104, 195, 339, 369, 392, 405, 459, 1, 3, 4,
                7, 12, 21, 39, 55, 62, 64, 113]))].copy(deep=True)
    output_frame['detection_classes'] = output_frame['detection_classes'].apply(lambda detection_class: "Nudity" if \
                                                                        detection_class in [32, 203, 284] else detection_class)
    output_frame['detection_classes'] = output_frame['detection_classes'].apply(lambda detection_class: "Car" if \
                                                                        detection_class in [8, 14, 16, 26, 88, 87, 93, 102, 
                                                                                107, 214, 251, 436, 446] else detection_class)
    output_frame['detection_classes'] = output_frame['detection_classes'].apply(lambda detection_class: "Weapon" if \
                                                                        detection_class in [104, 195, 339, 369, 392, 405, 
                                                                                            459] else detection_class)
    output_frame['detection_classes'] = output_frame['detection_classes'].apply(lambda detection_class: "Face" if \
                                                                        detection_class in [1, 3, 4, 7, 12, 21, 39, 55, 62,
                                                                                            64, 113] else detection_class)
    return output_frame

In [8]:
label_image(Path("__file__").parent.joinpath("images").joinpath("ghostgun-03.jpg"))

,detection_classes,detection_scores


# Downloading Benchmark

In [9]:
from google_images_download import google_images_download



response = google_images_download.googleimagesdownload()
arguments = {"keywords": "weapon", "limit": 200, "print_urls": False, "no_directory": True, "chromedriver": 
             "C:\\chromedriver\\chromedriver.exe", "output_directory": "weapons"}
response.download(arguments)


Item no.: 1 --> Item name = weapon
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1.destiny-2-best-weapons-braytech-lone-wolf-500x300-c.jpg
Completed Image ====> 2.Destiny_2_Reckoning_guns.0.jpg
Invalid or missing image format. Skipping...
Invalid or missing image format. Skipping...
Completed Image ====> 3.91TV8uvnwiL.jpg
Completed Image ====> 4.Fortnite_patch-notes_v6-22_overview-text-v6-22_BR06_Social_HeavyAR-1920x1080-c3f4eebedb39171525a13e8ca8af1d140f4135e6.jpg
Completed Image ====> 5.manhattan-weapon-gun-firearm-defense-lawyer.jpg
Completed Image ====> 6.960x0.jpg
Completed Image ====> 7.Destiny_2_Screenshot_2019.07.03___14.52.40.81.0.png
Completed Image ====> 8._106957772_mediaitem106957771.jpg
Completed Image ====> 9.prevweapon3-1024x538.jpg
Completed Image ====> 10.960x0.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 11.laser-china-1024x580.jpg
Completed Image ==

Completed Image ====> 125.a6ac04-Cover.jpg
Completed Image ====> 126.5cc74f23b14bf43f437ddea3-750-422.jpg
Completed Image ====> 127.guns_020317getty_0.jpg
Completed Image ====> 128.1369381658_43547.jpg
Completed Image ====> 129.fortnite-leaked-weapons-3.png
Invalid or missing image format. Skipping...
Completed Image ====> 130.image3.jpg
Completed Image ====> 131.mask-guns-rope-weapons-prince-albert.jpg
Completed Image ====> 132.Fun-Soft-Bullet-Gun-Toy-Kids-Electrical-Bursts-for-Nerf-Gun-Toy-Shooting-Submachine-Weapon-Pistol.jpg
Completed Image ====> 133.g7-scout-apex-legends.jpg
Completed Image ====> 134.destiny-2-night-watch.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 135.mw%20guns%205.png
Completed Image ====> 136.2841ecbd-ce55-4e93-8a85-6696ae842e4d-gunpile.jpg
Completed Image ====> 137.thumb_im_600x0_gallery_randysthrowingknife_5d8dcde1f0c6c.jpg
Completed Image ====> 138.EDjUnwdXkAsgFSN.jpg
Invalid or missing image format. Skipping...
Completed Image ===

({'weapon': ['C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\1.destiny-2-best-weapons-braytech-lone-wolf-500x300-c.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\2.Destiny_2_Reckoning_guns.0.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\3.91TV8uvnwiL.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\4.Fortnite_patch-notes_v6-22_overview-text-v6-22_BR06_Social_HeavyAR-1920x1080-c3f4eebedb39171525a13e8ca8af1d140f4135e6.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\5.manhattan-weapon-gun-firearm-defense-lawyer.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\6.960x0.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\7.Destiny_2_Screenshot_2019.07.03___14.52.40.81.0.png',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\weapons\\8._106957772_mediaitem106957771.jpg',
   'C:\\projects\\Object-Dete

In [10]:
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "nudity", "limit": 1000, "print_urls": False, "no_directory": True, "chromedriver": 
             "C:\\chromedriver\\chromedriver.exe", "output_directory": "nudity"}
response.download(arguments)


Item no.: 1 --> Item name = nudity
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1.160px-Anterior_view_of_human_female%2C_retouched_-_transparent.png
Completed Image ====> 2.1903.jpg
Completed Image ====> 3.naked-hot-nudist-enjoying-nudity-3(m=eaAaaEPbaaaa).jpg
Completed Image ====> 4.23-i-love-dick-devon-toby-feature-2.w512.h600.2x.jpg
Completed Image ====> 5.(m=eaf8Ggaaaa)(mh=jIGegnJHHI1-c4lM)1.jpg
Completed Image ====> 6.33221b1e327c6ef435a44b83f82b7d1f.18.jpg
Completed Image ====> 7.amazing-nudity-of-some-babes-on-the-beach-14(m=eaAaaEPbaaaa).jpg
Completed Image ====> 8.preview.mp4.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 9.334049.jpg
Completed Image ====> 10.161px-Anterior_view_of_human_male%2C_retouched%2C_cropped.jpg
Completed Image ====> 11.DV1ya4NX0AAKMzp.jpg
Completed Image ====> 12.(m=eafTGgaaaa)(mh=EbWd5LZb2RcJC3md)5.jpg
Completed Image ====> 13.hot-pub

Completed Image ====> 132.milo_ventimiglia_this_is_us_screenshot_h_2016.jpg
Completed Image ====> 133.peter_paul_rubens_-_the_toilet_of_venus_7.jpg
Completed Image ====> 134.voyeur-beach-nudity-with-hot-teen-babes-16(m=eaAaaEPbaaaa).jpg
Completed Image ====> 135.screen-shot-2018-03-28-at-5-17-40-pm.png
Completed Image ====> 136.Bob-Mizer-mens-nudity-bananas-w636-h600.png
IOError on an image...trying next one... Error: The read operation timed out
Invalid or missing image format. Skipping...
Completed Image ====> 137.jlaw.jpg
Completed Image ====> 138.Amistad.jpg
Completed Image ====> 139.Untitled-design-2-1-796x417.png
Completed Image ====> 140.OD5CYWPDU5FZXEWQYAW5AGTTFU.jpg
Completed Image ====> 141.beautiful-european-porno-star-enjoys-public-nudity.jpg
Completed Image ====> 142.il_340x270.1999630402_rvy6.jpg
Completed Image ====> 143.50-shades-of-grey.jpg
Completed Image ====> 144.Screen-Shot-2018-03-28-at-11.25.35-AM.png
Completed Image ====> 145.0dce29394d0a22ab7d452a0db22fada1fb52

Completed Image ====> 251.perfect-shot-of-pussy-and-asshole-nude-beach-pics-public-nudity.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 252.werethemillersnews.jpg
Completed Image ====> 253.lost-evangaline-lilly-main-everett-810x610.jpg
Completed Image ====> 254.GettyImages-160488757-e1503597287892-520x314.jpg
Completed Image ====> 255.s3-news-tmp-142968-lineker_crop--2x1--900.jpg
Completed Image ====> 256.Who-Plays-Daniel-Insecure.jpg
Completed Image ====> 257.tv-shows-with-male-nudity-640x480-1461095399.jpg
Completed Image ====> 258.sherlyn-and-poonam-promote-nudity.jpg
Completed Image ====> 259.nudity-lead-image.jpg
Completed Image ====> 260.02c06d43-5dda-446d-9c93-9311982a9903.jpg
Completed Image ====> 261.Dreamers-3-1024x500.jpg
URLError on an image...trying next one... Error: HTTP Error 403: Forbidden
Completed Image ====> 262.Amala-770x433.png
Completed Image ====> 263.russian-naked-wife-flashes-a-nudity-on-a-porn-amateur-milfs-selfies-15163994874kgn8.jpg

Completed Image ====> 371.isis03_03.jpg
Completed Image ====> 372.Nudity-in-Art-Michelangelo-and-More-Ingres-comparison-1.jpg
Completed Image ====> 373.GameOfThrones_Teaser06_Screencap_26.jpg
Completed Image ====> 374.attractive-nudity-woman-covering-maple-leafs-12196284.jpg
Completed Image ====> 375.call-me-by-your-name-slice-600x200.jpg
Completed Image ====> 376.Nudity.png
Completed Image ====> 377.nudity-closeup-isolated-sexy-model-600w-106003793.jpg
Completed Image ====> 378.fifty-shades-nudity-jamie-dornan.jpg
Completed Image ====> 379.star-trek-discovery-could-theoretically-involve-nudity-swearing-1-750x480.jpg
Completed Image ====> 380.big-soft-nudist-cock-800x475.jpg
Completed Image ====> 381.thrones-nudity_0.jpg
Completed Image ====> 382.Sherlock-Benedict-Cumberb-007.jpg
Completed Image ====> 383.34774274.jpg
Completed Image ====> 384.nudity_nakedness.png
Completed Image ====> 385.97438.jpg
Completed Image ====> 386.5d1393dcdda4c835188b45b5.png
Completed Image ====> 387.108959

URLError on an image...trying next one... Error: HTTP Error 403: Forbidden
Completed Image ====> 500.Nudity-in-Art-Michelangelo-and-More-Irving-Penn-comparison.jpg
Completed Image ====> 501.ike_barinholtz_3e4e69_infobox-6f270d2b_web.jpg
Completed Image ====> 502.13.jpg
Completed Image ====> 503.il_340x270.1922426623_7bg2.jpg
Completed Image ====> 504.nudity-in-art-hand-drawn-vector-4686955.jpg
Completed Image ====> 505.vitalia-curvy-nudity-david-nudes-1-683x1024.jpg
Completed Image ====> 506.35678efefaf2c4a4175dfc5d9a80ece14a258baf7b93fa7d20d4dd43ad26449d.jpg
Completed Image ====> 507.lisa-nolan-01-e1315515558722.jpg
Completed Image ====> 508.nudity7.jpg
Completed Image ====> 509._66226003_66167594.jpg
Completed Image ====> 510.in-darkness-1531159111.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 511.nudity-nipples-paint.jpg
Completed Image ====> 512.Shaz114-1200x675.jpg
Completed Image ====> 513.Sarah-Silverman-Topless-Breasts-Twitter-Instagram-Photo-Rules-860x

Completed Image ====> 624.220px-Nude_female_archer.jpg
Completed Image ====> 625.2661438-artsy-see-thru-nudity.jpg
Completed Image ====> 626.ozonnews.jpg
Completed Image ====> 627.50104398.jpg
Completed Image ====> 628.tv-show-survivor-nudity-uncensored.jpg
Completed Image ====> 629.41Y8Dxck9VL.jpg
Completed Image ====> 630.7b4b7077fbab3ec438afbd5659d99ae2.20.jpg
Completed Image ====> 631.Focus-actress-Margot-Robbie-nudity-Neighbours-Leonardo-DiCaprio-dating-Simon-Button-559392.jpg
Completed Image ====> 632.8f54cb5daeaff5cfee3264f6de40adf3488b1d75650edbf323f7b87c62332f12.jpg
Completed Image ====> 633.EBGogWWkBnjmFqG-400x400-noPad.jpg
Completed Image ====> 634.Nudity-in-Art-Michelangelo-and-More-Thomas-Kennington-comparison-1.jpg
Completed Image ====> 635.preview.jpg
Completed Image ====> 636.nudity-desire-for-facebook.jpg
Completed Image ====> 637.nudity-disable-girls.jpg
Completed Image ====> 638.got-nudity.jpg
Completed Image ====> 639.lena.png
Completed Image ====> 640.freepressjour

({'nudity': ['C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\1.160px-Anterior_view_of_human_female%2C_retouched_-_transparent.png',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\2.1903.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\3.naked-hot-nudist-enjoying-nudity-3(m=eaAaaEPbaaaa).jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\4.23-i-love-dick-devon-toby-feature-2.w512.h600.2x.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\5.(m=eaf8Ggaaaa)(mh=jIGegnJHHI1-c4lM)1.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\6.33221b1e327c6ef435a44b83f82b7d1f.18.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\7.amazing-nudity-of-some-babes-on-the-beach-14(m=eaAaaEPbaaaa).jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\nudity\\8.preview.mp4.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\

In [12]:
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "vehicles cars", "limit": 200, "print_urls": False, "no_directory": True, "chromedriver": 
             "C:\\chromedriver\\chromedriver.exe", "output_directory": "cars"}
response.download(arguments)


Item no.: 1 --> Item name = vehicles cars
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1.nav-outlander-20my.png
Completed Image ====> 2.test-own-car.png
Completed Image ====> 3.harrier-pv-landing-banner-mob.jpg
Completed Image ====> 4.626962-mercedes-benz-vision-urbanetic.jpg
Completed Image ====> 5.maxresdefault.jpg
Completed Image ====> 6.TOYO_CAMR_2018.png
Completed Image ====> 7.CHEV_MALI_2016.png
URLError on an image...trying next one... Error: HTTP Error 403: Forbidden
Completed Image ====> 8.cars---do-not-delete-473727.jpg
Completed Image ====> 9.hyundai-kona-electric_360x240.jpg
Completed Image ====> 10.CAC80NIC141A021001.png
Completed Image ====> 11.CCAR_800x400.png
Completed Image ====> 12.25-cars-worth-waiting-for-lp-ford-gt-photo-658253-s-original.jpg
Completed Image ====> 13.ICAR_800x400.png
Completed Image ====> 14.Rolls_Royce_Phantom_IV_1952.JPG
Completed Image ====> 15.model_s_

Completed Image ====> 119.mitsubishi-2019-eclipse-cross-d.jpg
Completed Image ====> 120.pcna18-0239-fine-1547659271.jpg
Completed Image ====> 121.eq-nav-pure.png
Completed Image ====> 122.6-1920x720.jpg
Completed Image ====> 123.homepage-genius-miata-car.png
Completed Image ====> 124.mercedes2.jpg
Completed Image ====> 125.2019-q3-electrified-vehicles-1920x1080.jpg
Completed Image ====> 126.BMW-Concept-s-4SeriesCoupe.png
Completed Image ====> 127.8b1c41ae758b263a864ca709d95c50f7378d5f98.jpg
Completed Image ====> 128.102675094-471432770.jpg
Completed Image ====> 129.$_35.JPG
Completed Image ====> 130.xmercedes_benz_a_class_hatchback_a180_sport_executive_5dr_2018_front_three_quarter.jpeg.pagespeed.ic.ZZR7lPT_NA.jpg
Completed Image ====> 131.6023177_05722_CAS_3MZBN1L71JM232546_-1888A_2018_Mazda_Mazda3-Sport_USED-01.jpg
Completed Image ====> 132.d6f78f3ec4c8c3723360e076c71b4258x.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 133.d1ec66195b874f589dee2bacda151e1b_c0x0

({'vehicles cars': ['C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\1.nav-outlander-20my.png',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\2.test-own-car.png',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\3.harrier-pv-landing-banner-mob.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\4.626962-mercedes-benz-vision-urbanetic.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\5.maxresdefault.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\6.TOYO_CAMR_2018.png',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\7.CHEV_MALI_2016.png',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\8.cars---do-not-delete-473727.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\9.hyundai-kona-electric_360x240.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\cars\\10.CAC80NIC141A021001.png',
   'C:\\pro

In [13]:
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "drugs marijuana", "limit": 200, "print_urls": False, "no_directory": True, "chromedriver": 
             "C:\\chromedriver\\chromedriver.exe", "output_directory": "drugs"}
response.download(arguments)


Item no.: 1 --> Item name = drugs marijuana
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1.marijuana-from-marijuana-farm.jpg
Completed Image ====> 2.joints_shutterstock-52960849.jpg
Completed Image ====> 3.157923419.jpg
Completed Image ====> 4.lead_720_405.jpg
URLError on an image...trying next one... Error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'michiganradionews.org'. (_ssl.c:1076)>
Invalid or missing image format. Skipping...
Completed Image ====> 5.c35a1265d4389cb6f274999af04c50db.jpg
Completed Image ====> 6.marijuanaleaf_article.jpg
Completed Image ====> 7.Hemp-cannabis-37601347-2.jpg
Completed Image ====> 8.MJardin.jpg
Completed Image ====> 9.shutterstock_345007262-e1495731023304.jpg
Completed Image ====> 10.123916503-cannabis-drugs-marijuana-leaf-closeup-on-the-white-background-.jpg
Invalid or missing i

Completed Image ====> 105.102807674-don-t-do-drugs-sign-hand-drawn-marijuana-leaf-and-red-brush-stroke-stop-narcotic-cannabis-prohibited.jpg
Completed Image ====> 106.934eacbe5cde7c0575d6bf38174b-is-marijuana-really-a-gateway-drug.jpg
Completed Image ====> 107.2502x2043-px-drugs-marijuana-weed-805738.jpg
Completed Image ====> 108.2134a198785ee3d610d09f1e8c6f9712-700.jpg
Completed Image ====> 109.Mixing-marijuana-with-prescription-drugs-can-be-dangerous-624x417.jpg
Completed Image ====> 110.marijuana.jpg
Completed Image ====> 111.drugs-marijuana-high-resolution-widescreen-background-images-weed-beautiful.jpg
Completed Image ====> 112.TIMELINE_marijuana-01.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 113.Marijuana.jpg
Completed Image ====> 114.75tn.gif
Completed Image ====> 115.1012309-weed-drugs-marijuana-420-nature-psychedelic-plant-cannabis-rasta-reggae-drug-trippy-748x421.jpg
Completed Image ====> 116.marijuana-gateway-drug.jpg
Completed Image ====> 117.8180

({'drugs marijuana': ['C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\1.marijuana-from-marijuana-farm.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\2.joints_shutterstock-52960849.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\3.157923419.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\4.lead_720_405.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\5.c35a1265d4389cb6f274999af04c50db.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\6.marijuanaleaf_article.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\7.Hemp-cannabis-37601347-2.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\8.MJardin.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\9.shutterstock_345007262-e1495731023304.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\drugs\\10.123916503-cannabis

In [14]:
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "upskirt", "limit": 1000, "print_urls": False, "no_directory": True, "chromedriver": 
             "C:\\chromedriver\\chromedriver.exe", "output_directory": "upskirt"}
response.download(arguments)


Item no.: 1 --> Item name = upskirt
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1.hottest-public-upskirt-video-ever-13(m=eaAaaEPbaaaa).jpg
Completed Image ====> 2.public-street-upskirt-3(m=eaAaaEPbaaaa).jpg
Completed Image ====> 3.5557965-upskirt-on-the-balcony.jpg
Completed Image ====> 4.georgia-steel-080218a.jpg
Completed Image ====> 5.1e5a93.jpg
Completed Image ====> 6.Amanda-Seyfried-Paris-upskirt-HD.jpg
Completed Image ====> 7.upskirtonly-sharlotte-ftvgirls.jpg
Completed Image ====> 8.7.jpg
Completed Image ====> 9.2654958.jpg
Completed Image ====> 10.538772_06big.jpg
Completed Image ====> 11.5.jpg
Completed Image ====> 12.b8f5978ef.jpg
Completed Image ====> 13.19.jpg
Completed Image ====> 14.21f7c19cd.jpg
Completed Image ====> 15.125080_04.jpg
Completed Image ====> 16.20180312133030uid1.jpg
Completed Image ====> 17.main.jpg
Completed Image ====> 18.pMzlTka.jpg
Completed Image ====> 19.17

Completed Image ====> 158.8.jpg
Completed Image ====> 159.900803.jpg
Completed Image ====> 160.Tamara-Beckwith-Celebrity-Upskirt-Pussy-Slip.jpg
Completed Image ====> 161.59e69266175a5.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 162.48a7c4bad.jpg
Completed Image ====> 163.Upskirt-panties.jpg
Completed Image ====> 164.Marcia_Imperator_legs.jpg
Completed Image ====> 165.upskirt-opportunity-preview-small.jpg
Completed Image ====> 166.281026_05.jpg
Completed Image ====> 167.upskirtonly-bella-ftvgirls.jpg
Completed Image ====> 168.main.jpg
Completed Image ====> 169.6216604.jpg
Completed Image ====> 170.a426d565_big.jpg
Completed Image ====> 171.50d2f0a105e9b747abfc944a35e316fe.jpg
Completed Image ====> 172.price-is-right-upskirt.jpg
Completed Image ====> 173.preview.jpg
Completed Image ====> 174.(m=eafTGgaaaa)(mh=cU0QXspr3e69FROp)3.jpg
Completed Image ====> 175.upskirt-girls-fuck-2.jpg
Completed Image ====> 176.th270x360_ursula_hall_06.jpg
Completed Image ====> 177

Completed Image ====> 312.565b302635356bab7e41ab7c175090f5.jpg
Completed Image ====> 313.3170566.jpg
Completed Image ====> 314.1312376463994.jpg
Completed Image ====> 315.1062480.jpg
Completed Image ====> 316.smallimage5.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 317.140040.jpg
Completed Image ====> 318.6152-04.jpg
Completed Image ====> 319.preview.mp4.jpg
Completed Image ====> 320.4.jpg
Completed Image ====> 321.Public-upskirt-pics-accidental-ass-flash.jpg
Completed Image ====> 322.637.jpg
Completed Image ====> 323.sitting+Upskirt+no+Undies.jpg
Completed Image ====> 324.512556_07.jpg
Completed Image ====> 325.31234986.jpg
Completed Image ====> 326.6671661.jpg
Completed Image ====> 327.5.jpg
Completed Image ====> 328.7.jpg
Completed Image ====> 329.261-blonde-upskirt-over.jpg
Completed Image ====> 330.85893.jpg
Completed Image ====> 331.Demi-Rose-Upskirt-707.jpg
Completed Image ====> 332.black-shemale-upskirt-huge-anal-sex-toys.jpg
Completed Image ====> 333.

Completed Image ====> 473.585.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 474.29913159.jpg
Completed Image ====> 475.black-tgirl-upskirt-0104.jpg
Completed Image ====> 476.170131.jpg
Completed Image ====> 477.6.jpg
Completed Image ====> 478.Kourtney-Kardashian-Upskirt-Panties-Halloween.jpg
Completed Image ====> 479.upskirt-flashing-thumbs.jpg
Completed Image ====> 480.e7cb599e8.jpg
Completed Image ====> 481.4c94771f26.jpg
Completed Image ====> 482.45938-Sririta-Jensen-Upskirt-2-TheFappeningTop.com_-748x421.jpg
Completed Image ====> 483.963_Masturbates_Public.jpg
Completed Image ====> 484.1.jpg
Completed Image ====> 485.532_Image_gallery.jpg
Completed Image ====> 486.Samantha-Capitoni-Upskirt-2.jpg
Completed Image ====> 487.Bh_443913.jpg
Completed Image ====> 488.167dacdb34.jpg
Completed Image ====> 489.1123930681.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 490.5.jpg
Completed Image ====> 491.georgia-harrison-082719-300x473.jpg
Compl

Completed Image ====> 626.762_UPSKIRT_UPSKIRT.jpg
Invalid or missing image format. Skipping...
Completed Image ====> 627.nikki-reed-upskirt-panty-peek-111011.jpg
Completed Image ====> 628.538772_01big.jpg
Completed Image ====> 629.9.jpg
Completed Image ====> 630.Courteney+Cox+Sheer+Panty+Upskirt+www.GutterUncensored.com+002.jpg
Completed Image ====> 631.Lindsey-Vonn-Upskirt-4.jpg
Completed Image ====> 632.3ba4977b4505d92d4c7db3427c792dc2.jpg
Completed Image ====> 633.(m=eaAaGwObaaaa)(mh=nuJ1D26slU_C4CA0)4.jpg
Completed Image ====> 634.16.jpg
Completed Image ====> 635.Celebrities-Upskirt-Exposed-by-Accidents.jpg
Completed Image ====> 636.3576814_3_p.jpg
Completed Image ====> 637.giphy.gif
Completed Image ====> 638.1537796-upskirt-view.jpg
Completed Image ====> 639.51VHF%2B1TveL._SX385_.jpg
Completed Image ====> 640.3.jpg
Completed Image ====> 641.102108554b8ee10db065.jpg
Completed Image ====> 642.942715.jpg
Completed Image ====> 643.8.jpg
Completed Image ====> 644.Candid-Teen-Upskirt-Wi

Completed Image ====> 776.hairy-pussy-upskirt-pov-pussy-upskirt-hairy-upskirt-hairy-upskirt-porn-hairy-upskirt-jpg.jpg
Completed Image ====> 777.sexy-vr-upskirt-horny-facesitting-1530167377.jpg
Completed Image ====> 778.katy_perry_upskirt_pussy_slip.jpg
Completed Image ====> 779.no-porn-older-upskirt-pics-2.jpg
Completed Image ====> 780.Taylor-Swift-Upskirt-TheFappeningBlog.com-21-1024x1536.jpg
Completed Image ====> 781.2.jpg
Completed Image ====> 782.3695022_11_p.jpg
Completed Image ====> 783.349897.jpg
URLError on an image...trying next one... Error: HTTP Error 403: Forbidden
Completed Image ====> 784.1578001-upskirt-in-park.jpg
Completed Image ====> 785.22.jpg
Completed Image ====> 786.828034.jpg
Completed Image ====> 787.6417.jpg
Completed Image ====> 788.588632_04big.jpg
Completed Image ====> 789.5510749.jpg
Completed Image ====> 790.308779031.jpg
Completed Image ====> 791.b1687242fba1cbfff5ed09cb68e988de.7.jpg
Completed Image ====> 792.ta%20%289%29_m.jpg
Completed Image ====> 793

({'upskirt': ['C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\1.hottest-public-upskirt-video-ever-13(m=eaAaaEPbaaaa).jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\2.public-street-upskirt-3(m=eaAaaEPbaaaa).jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\3.5557965-upskirt-on-the-balcony.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\4.georgia-steel-080218a.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\5.1e5a93.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\6.Amanda-Seyfried-Paris-upskirt-HD.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\7.upskirtonly-sharlotte-ftvgirls.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\8.7.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\\upskirt\\9.2654958.jpg',
   'C:\\projects\\Object-Detection-Experiments\\experiments\